In [87]:
import pandas as pd
import cv2
import imageio
import matplotlib
import os
import os.path
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from datetime import date, timedelta
import datetime as dt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import random
import ast
import gc
import time

user_name = " " #change the user name accordingly
basepath = " " % user_name # insert path accordingly
exportpath = " " % user_name #delete/modify after jupyter-%s as needed

start = time.time()
df_magview = pd.read_csv(basepath + "/magview_all_cohorts_anon_HITI.csv")
df_magview = df_magview[['empi_anon','acc_anon','study_date_anon','asses',
                         'desc','side','path_severity','numfind','numfind_unique']]

df_metadata = pd.read_csv(basepath + "/metadata_all_cohort_with_ROI_HITI.csv")
df_metadata = df_metadata[['png_path','empi_anon','acc_anon','study_date_anon',
                            'StudyDescription','SeriesDescription','FinalImageType',
                            'ImageLateralityFinal','ViewPosition','spot_mag','ROI_coords']]


/home/jupyter-lzha327/.conda/envs/echolinglin/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (28,31,33,48,49,50,51,53,54,55,56,57,59,61,63,76,78,81,84,85,93) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jupyter-lzha327/.conda/envs/echolinglin/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (75,104,105,114,115,116,117,118,119,129,131,132,133,134,135,137,142,146,147,148,150,156) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [88]:
df_magview_br0 = df_magview[df_magview.asses == "A"]
df_magview = pd.DataFrame()
gc.collect()
df_magview_br0 = df_magview_br0.drop_duplicates()

In [89]:
df_magview_br0 = df_magview_br0.drop_duplicates(subset = ['acc_anon', 'side'],
                            keep = 'last').reset_index(drop = True)
df_magview_br0

,empi_anon,acc_anon,study_date_anon,asses,desc,side,path_severity,numfind,numfind_unique
0,46004361,1690271690233973,2013-07-25 00:00:00,A,MG Screening Bilateral w/CAD,L,4.0,2,NaN
1,46004361,1690271690233973,2013-07-25 00:00:00,A,MG Screening Bilateral w/CAD,R,NaN,3,NaN
2,90448002,9534293829709675,2013-05-14 00:00:00,A,MG Screening Bilateral w/CAD,L,NaN,1,NaN
3,87644551,9016910987969796,2012-12-17 00:00:00,A,MG Screening Bilateral w/CAD,L,NaN,1,NaN
4,62668630,2274583860519142,2013-03-17 00:00:00,A,MG Screening Bilateral w/CAD,R,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...
53831,68040298,8654675989685565,2019-10-28 00:00:00,A,MG Screen Bilat w/Tomo/CAD Stnd Protocol,R,NaN,1,NaN
53832,68040298,8654675989685565,2019-10-28 00:00:00,A,MG Screen Bilat w/Tomo/CAD Stnd Protocol,B,NaN,2,NaN
53833,45287126,5333144471536352,2020-08-06 00:00:00,A,MG Screen Bilat w/Tomo/CAD Stnd Protocol,B,NaN,1,NaN
53834,50146147,5807859000538017,2021-02-06 00:00:00,A,MG Screen Bilat w/Tomo/CAD Stnd Protocol,R,NaN,2,NaN


In [90]:
df_magview_br0.path_severity.value_counts()

4.0    3663
0.0     950
2.0     870
1.0     531
5.0      60
3.0      33
Name: path_severity, dtype: int64

In [91]:
df_magview_br0_cancer = df_magview_br0[df_magview_br0.path_severity.isin([0,1])].drop(columns=['path_severity','numfind','numfind_unique'])
df_magview_br0_benign = df_magview_br0[df_magview_br0.path_severity.isin([2,3,4,5])].drop(columns=['path_severity','numfind','numfind_unique'])
df_magview_br0_unknown = df_magview_br0[np.isnan(df_magview_br0.path_severity)].drop(columns=['path_severity','numfind','numfind_unique'])


In [94]:
df_magview_br0_cancer = df_magview_br0_cancer.drop_duplicates()
print(df_magview_br0_cancer.shape)
df_magview_br0_benign = df_magview_br0_benign.drop_duplicates()
print(df_magview_br0_benign.shape)
df_magview_br0_unknown = df_magview_br0_unknown.drop_duplicates()
print(df_magview_br0_unknown.shape)

(1481, 6)
(4626, 6)
(47729, 6)


In [86]:
#Getting metadata dataset ready
br0_pt = list(df_magview_br0.acc_anon.unique())
df_meta_br0 = df_metadata[df_metadata.acc_anon.isin(br0_pt)]
df_metadata = pd.DataFrame()
gc.collect()
df_meta_br0_2d = df_meta_br0[df_meta_br0.FinalImageType == '2D']
df_meta_br0_2d

,png_path,empi_anon,acc_anon,study_date_anon,StudyDescription,SeriesDescription,FinalImageType,ImageLateralityFinal,ViewPosition,spot_mag,ROI_coords
21,/data/mammo/png_kheiron/kheiron_code/cohort_1/...,66685715,6440018351487332,2015-05-11 00:00:00,MG Screening Bilateral,RCC,2D,R,NaN,NaN,()
29,/data/mammo/png_kheiron/kheiron_code/cohort_1/...,66685715,6440018351487332,2015-05-11 00:00:00,MG Screening Bilateral,RMLO,2D,R,NaN,NaN,()
33,/data/mammo/png_kheiron/kheiron_code/cohort_1/...,66685715,6440018351487332,2015-05-11 00:00:00,MG Screening Bilateral,LMLO,2D,L,NaN,NaN,()
43,/data/mammo/png_kheiron/kheiron_code/cohort_1/...,66685715,6440018351487332,2015-05-11 00:00:00,MG Screening Bilateral,LCC,2D,L,NaN,NaN,()
174,/data/mammo/png_kheiron/kheiron_code/cohort_1/...,41364831,1408943148887891,2017-07-10 00:00:00,MG Screen Bilat w/Tomo/CAD Stnd Protocol,R MLO,2D,R,MLO,NaN,()
...,...,...,...,...,...,...,...,...,...,...,...
3383616,/data/mammo/png_kheiron/kheiron_code/cohort_10...,86898110,9286291546086342,2015-03-16 00:00:00,MG Screen Bilat w/Tomo/CAD Stnd Protocol,R CC,2D,R,CC,NaN,()
3383636,/data/mammo/png_kheiron/kheiron_code/cohort_10...,86898110,9286291546086342,2015-03-16 00:00:00,MG Screen Bilat w/Tomo/CAD Stnd Protocol,L MLO,2D,L,MLO,NaN,()
3383638,/data/mammo/png_kheiron/kheiron_code/cohort_10...,86898110,9286291546086342,2015-03-16 00:00:00,MG Screen Bilat w/Tomo/CAD Stnd Protocol,L CC,2D,L,CC,NaN,()
3383639,/data/mammo/png_kheiron/kheiron_code/cohort_10...,86898110,9286291546086342,2015-03-16 00:00:00,MG Screen Bilat w/Tomo/CAD Stnd Protocol,R MLO,2D,R,MLO,NaN,()


In [96]:
cmerge1 = pd.merge(df_meta_br0_2d, df_magview_br0_cancer,on=['acc_anon'])
bmerge1 = pd.merge(df_meta_br0_2d, df_magview_br0_benign,on=['acc_anon'])
umerge1 = pd.merge(df_meta_br0_2d, df_magview_br0_unknown,on=['acc_anon'])


#Filter to keep only the rows where the side matches
cmerge1 = cmerge1[
    (cmerge1.side == cmerge1.ImageLateralityFinal) 
    | (cmerge1.side == 'B') 
    | (cmerge1.ImageLateralityFinal == 'B')]
cmerge1 = cmerge1.drop_duplicates()

bmerge1 = bmerge1[
    (bmerge1.side == bmerge1.ImageLateralityFinal) 
    | (bmerge1.side == 'B') 
    | (bmerge1.ImageLateralityFinal == 'B')]
bmerge1 = bmerge1.drop_duplicates()

umerge1 = umerge1[
    (umerge1.side == umerge1.ImageLateralityFinal) 
    | (umerge1.side == 'B') 
    | (umerge1.ImageLateralityFinal == 'B')]
umerge1 = umerge1.drop_duplicates()


print(cmerge1.shape)
print(bmerge1.shape)
print(umerge1.shape)

(3490, 16)
(11503, 16)
(125677, 16)


In [98]:
cmerge1.to_csv(exportpath+'/01df_Cancer_2D.csv',index=False)
bmerge1.to_csv(exportpath+'/01df_Benign_2D.csv',index=False)
umerge1.to_csv(exportpath+'/01df_Unknown_2D.csv',index=False)

In [97]:
print(len(cmerge1.empi_anon_x.unique()))
print(len(bmerge1.empi_anon_x.unique()))
print(len(umerge1.empi_anon_x.unique()))

1278
3928
35956
